In [1]:
import csv
import random
import pandas as pd
import numpy as np
import os
import seaborn as sns
from time import time
import matplotlib.pyplot as plt
import scipy.io
import glob

In [2]:
input_folder = 'test/'
output_file = 'output_file.csv'

excel_files = glob.glob(os.path.join(input_folder, '*.xlsx'))
#xcel_files = glob.glob(os.path.join(input_folder, '*', '*.xlsx'), recursive=True)
df_list = []

for input_file in excel_files:
    df = pd.read_excel(input_file)

    df = df.iloc[4:].reset_index(drop=True)
    df.columns = ['Row', 'Time', 'GPS-Validity', 'latitude', 'longitude', 'speed', 'engine-status', 'Received Date','9', '10' ]

    df = df.drop(index=0).reset_index(drop = True)
    df_list.append(df)

  
final_df = pd.concat(df_list, ignore_index=True)
pd.set_option('future.no_silent_downcasting', True)
final_df['GPS-Validity'] = final_df['GPS-Validity'].replace({'بلی': 1, 'خیر': 0})



#final_df.to_csv(output_file, index=False)

In [3]:
for column in final_df.columns:
    if final_df[column].isna().all():
        print(f" entries in '{column}' are NaN.")

 entries in '9' are NaN.


In [4]:
if final_df['9'].isna().all():
    final_df = final_df.drop(columns=['9'])


In [5]:
final_df = final_df.drop(columns=['Row', 'Received Date'])


In [6]:
final_df['10'] = final_df['10'].fillna(0)

In [7]:
final_df = final_df.rename(columns = {'10': 'label'})
final_df['label'] = final_df['label'].astype(int)

In [8]:
value_counts = final_df['engine-status'].value_counts()
value_counts

engine-status
on     164
off     89
Name: count, dtype: int64

In [9]:
mapping = {
    'off': 0,
    'on': 1,
    '-': -1
}

pd.set_option('future.no_silent_downcasting', True)
final_df['engine-status'] = final_df['engine-status'].replace(mapping)

In [10]:
final_df

,Time,GPS-Validity,latitude,longitude,speed,engine-status,label
0,1403/03/13 11:06:51,1,35.755108,51.549366,27,1,0
1,1403/03/13 11:07:50,1,35.755108,51.549366,0,1,0
2,1403/03/13 11:07:51,1,35.755108,51.549366,0,0,0
3,1403/03/13 11:13:24,1,35.755108,51.549366,0,1,0
4,1403/03/13 11:13:25,1,35.755108,51.549366,0,1,0
...,...,...,...,...,...,...,...
248,1403/03/13 17:49:48,1,35.758846,51.550503,0,0,0
249,1403/03/13 18:04:59,1,35.758751,51.550587,0,0,0
250,1403/03/13 18:19:59,1,35.758774,51.550667,0,0,0
251,1403/03/13 18:34:59,1,35.758652,51.550678,0,0,0


In [13]:
final_df.to_csv(output_file, index=False)

In [11]:
split_columns = final_df['Time'].str.split(' ', n=1, expand=True)


final_df['Date'] = split_columns[0]
final_df['t'] = split_columns[1]

In [12]:
import jdatetime
from datetime import datetime
def to_gregorian(persian_date_str, strftime):
    if pd.isna(persian_date_str):
        return '-'
    try:
        # Parse the Persian date string to Jalali date
        jalali_date = jdatetime.datetime.strptime(persian_date_str, '%Y/%m/%d')
        # Convert to Gregorian and format the string
        gregorian_date = jalali_date.togregorian()
        return gregorian_date.strftime(strftime)
    except ValueError:
        return '-'

# Convert 'Date' column from Persian to Gregorian
final_df['Date'] = final_df['Date'].apply(lambda value: to_gregorian(value, '%Y-%m-%d'))

In [13]:
final_df

,Time,GPS-Validity,latitude,longitude,speed,engine-status,label,Date,t
0,1403/03/13 11:06:51,1,35.755108,51.549366,27,1,0,2024-06-02,11:06:51
1,1403/03/13 11:07:50,1,35.755108,51.549366,0,1,0,2024-06-02,11:07:50
2,1403/03/13 11:07:51,1,35.755108,51.549366,0,0,0,2024-06-02,11:07:51
3,1403/03/13 11:13:24,1,35.755108,51.549366,0,1,0,2024-06-02,11:13:24
4,1403/03/13 11:13:25,1,35.755108,51.549366,0,1,0,2024-06-02,11:13:25
...,...,...,...,...,...,...,...,...,...
248,1403/03/13 17:49:48,1,35.758846,51.550503,0,0,0,2024-06-02,17:49:48
249,1403/03/13 18:04:59,1,35.758751,51.550587,0,0,0,2024-06-02,18:04:59
250,1403/03/13 18:19:59,1,35.758774,51.550667,0,0,0,2024-06-02,18:19:59
251,1403/03/13 18:34:59,1,35.758652,51.550678,0,0,0,2024-06-02,18:34:59


In [14]:
final_df['timestamp'] = pd.to_datetime(
    final_df['Date'] + ' ' +
    final_df['t'].astype(str)
)

# final_df = final_df.sort_values('timestamp').reset_index(drop=True)

In [30]:
# final_df[['Year', 'Month', 'Day']] = final_df['Date'].str.split('-', expand=True)
# final_df[['Hour', 'Minute', 'Second']] = final_df['t'].str.split(':', expand=True)

# final_df['Year'] = final_df['Year'].astype(int)
# final_df['Month'] = final_df['Month'].astype(int)
# final_df['Day'] = final_df['Day'].astype(int)


# final_df['Hour'] = final_df['Hour'].astype(int)
# final_df['Minute'] = final_df['Minute'].astype(int)
# final_df['Second'] = final_df['Second'].astype(int)

In [17]:
final_df = final_df.drop(columns=['Time', 'Date', 't'])


In [18]:
final_df

,GPS-Validity,latitude,longitude,speed,engine-status,label,timestamp
0,1,35.755108,51.549366,27,1,0,2024-06-02 11:06:51
1,1,35.755108,51.549366,0,1,0,2024-06-02 11:07:50
2,1,35.755108,51.549366,0,0,0,2024-06-02 11:07:51
3,1,35.755108,51.549366,0,1,0,2024-06-02 11:13:24
4,1,35.755108,51.549366,0,1,0,2024-06-02 11:13:25
...,...,...,...,...,...,...,...
248,1,35.758846,51.550503,0,0,0,2024-06-02 17:49:48
249,1,35.758751,51.550587,0,0,0,2024-06-02 18:04:59
250,1,35.758774,51.550667,0,0,0,2024-06-02 18:19:59
251,1,35.758652,51.550678,0,0,0,2024-06-02 18:34:59


In [19]:
is_all_nan = final_df.isna().all().sum()

is_all_nan

0